# Tutorial-03 Intro to lateral Stanley control 

Control is the lowest layer of the entire unmanned platform system, accepting the planned path and speed information and executing it. It can be further subdivided into lateral control and longitudinal control, corresponding to the operations of stepping on the throttle/brake and turning the steering wheel in the actual driving process. The subsequent courses will attempt to control and plan from the bottom up. If time permits, perception and positioning will be introduced.

The name of the Stanley lateral control algorithm comes from the "Stanley" team, the champion of the 2005 DARPA Challenge from Stanford University. It is a non-linear feedback function based on the lateral tracking error as the distance from the center of the front axle to the nearest path point, and it can realize the exponential convergence of the lateral tracking error to 0. According to the relative geometric relationship between the vehicle pose and the given path, the control variable for controlling the steering wheel turning angle of the vehicle can be intuitively obtained.

The front wheel steering angle control variable consists of two parts: one part is the steering angle caused by the heading error, that is, the angle between the current vehicle body direction and the tangent direction of the nearest point of the reference trajectory; the other part is the steering angle caused by the lateral error, that is, the lateral distance from the center of the front wheel to the nearest point of the reference trajectory.

<img src="./media/stanley.png" width="420" height="525">

We define the reference point $P_1$ as the point closest to the center of the vehicle's front wheel. There is a point $P_2$ at a distance of $d(t)$ in front of the tangent line of point $P_1$. The basic idea of the Stanley method is to reduce the lateral error $e_y$ and the heading error $\theta_y$.

**Without considering lateral errors**, the front wheel should be in line with the tangent direction of the given path reference point. Here, $\theta_\varphi$ represents the angle between the vehicle heading and the tangent direction of the nearest path point. In the absence of any lateral errors, the front wheel steering angle $\delta_\varphi$ is the same direction as the direction of the path point where it is located.
$$
\delta_\varphi = \theta_\varphi
$$

**Without considering the heading error**, the larger the lateral tracking error, the larger the front wheel steering angle. Assuming that the vehicle's expected trajectory intersects with the tangent line of the nearest point on the given path at a distance $d(t)$ from the front wheel. According to the geometric relationship, the following nonlinear proportional function is derived:
$$
\delta_y = \theta_y = \arctan \frac{e_y(t)}{d(t)} = \arctan \frac{ke_y(t)}{v(t)}, \quad d(t) = \frac{1}{k}v(t)
$$
where $d(t)$ is related to the vehicle speed, represented by the vehicle speed $v(t)$ and the gain parameter $k$, $d(t) = ke_y(t)$.

Considering both the lateral error and the heading error, the control quantity can be obtained:
$$
\delta = \delta_\varphi + \delta_y
$$

In the formula, there is an important parameter -- the lateral error $e(y)$. The positive and negative of the lateral error determine the position of the vehicle relative to the road. Its calculation method can be determined by the geometric relationship in the following figure:

<img src="./media/latError.png" width="470" height="525">

$$
\begin{align}
d_y &= y_r - y \\
d_x &= x_r - x \\
\vec{n}_r &= (-\sin\theta, \cos\theta) \\
e_y &= (\vec{X} - \vec{X}_r) \cdot \vec{n}_r \\
    &= d_y \cos \theta - d_x \sin\theta
\end{align}
$$

In [1]:
import math

'''
Normalize the angle in rad to [-pi, pi]
'''
def normalize_angle(angle):
    normalized_angle = math.fmod(angle + math.pi, 2.0 * math.pi)
    
    if (normalized_angle < 0.0):
        normalized_angle = normalized_angle + 2.0 * math.pi
    
    return normalized_angle - math.pi

'''
A Struct that stores the position and heading of ego vehicle
'''
class Transform:
    def __init__(self, x, y, theta):
        self.x_     = x
        self.y_     = y
        self.theta_ = normalize_angle(theta)

'''
A Struct that stores the state of ego vehicle
'''
class State:
    def __init__(self, pose: Transform, v):
        self.pose_ = pose
        self.v_    = v
        
'''
The Stanley Controller
'''
class Stanley:
    def __init__(self, k):
        self.k_ = k # Initialize preview coefficient
        
    def update_control(self, P1: Transform, ego: State):
        # Calculate lateral error and relative control command
        # TODO     
        delta_y = 0
        
        # Calculate yaw error and relative control command
        # TODO
        delta_theta = 0
        
        # Calculate control command
        delta = delta_y + delta_theta
        
        return delta 

The kinematic model of a vehicle is a simplification of its real-world dynamics that assumes there are no forces such as traction, drag, or weight distribution affecting the vehicle. It's a useful model for many autonomous driving applications where these forces are negligible or can be approximated well enough for the task at hand.

The state of the vehicle at any given time is described by its position $(x_k, y_k)$, its heading $\theta_k$, and its velocity $v_k$. These are the variables that we update in the equations of the kinematic model.

The position of the vehicle is updated based on its current velocity and heading. If the vehicle is moving at a velocity $v_k$ and its heading is $\theta_k$, the horizontal and vertical components of the distance it covers in a small time step $dt$ are $v_k \cos(\theta_k) dt$ and $v_k \sin(\theta_k) dt$, respectively. These distance components are added to the current position $(x_k, y_k)$ to get the new position $(x_{k+1}, y_{k+1})$:

$$
\begin{align}
    x_{k+1} &= x_k + v_k \cos(\theta_k) \cdot dt \\
    y_{k+1} &= y_k + v_k \sin(\theta_k) \cdot dt
\end{align}
$$

The heading of the vehicle is updated based on the vehicle's current velocity, the wheelbase $L$, and the steering angle $\delta_k$. The change in heading over a small time step $dt$ is $\frac{v_k}{L} \tan(\delta_k) dt$, which is added to the current heading $\theta_k$ to get the new heading $\theta_{k+1}$:

$$
\theta_{k+1} = \theta_k + \frac{v_k}{L} \tan(\delta_k) \cdot dt
$$

The velocity of the vehicle is updated based on its current acceleration $a_k$. The change in velocity over a small time step $dt$ is $a_k dt$, which is added to the current velocity $v_k$ to get the new velocity $v_{k+1}$:

$$
v_{k+1} = v_k + a_k \cdot dt
$$

In these updates, the wheelbase $L$ is a constant that represents the distance between the front and rear wheels of the vehicle, and $dt$ is the time step size that we're simulating.

In [2]:
'''
Vehicle Kinematic Model
'''
class Vehicle:
    def __init__(self, L, initial_state: State):
        self.L_ = L
        self.x_ = initial_state.pose_.x_
        self.y_ = initial_state.pose_.y_
        self.theta_ = normalize_angle(initial_state.pose_.theta_)
        self.v_ = initial_state.v_

    def update_state(self, delta, a, dt):
        # Complete the vehicle kinematic model
        # TODO

        new_pose  = Transform(self.x_, self.y_, self.theta_)
        new_state = State(new_pose, self.v_)
         
        return new_state

Don't forget the PID controller in previous tutorial.

In [3]:
'''
the PID controller
'''
class PidController:
    previous_error_ = 0 # Control error on previous iteration
    accum_error_    = 0 # Accumulate error
    
    def __init__(self, Kp, Ki, Kd):
        self.Kp_ = Kp # Initialize proportion gain
        self.Ki_ = Ki # Initialize integral gain
        self.Kd_ = Kd # Initialize derivative gain
        
    def update_control(self, target_v, current_v, dt):
        # Calculate proportion term
        error = target_v - current_v
        proportion_term = self.Kp_ * error
        
        # Calculate integral term
        self.accum_error_ += error
        integral_term = self.Ki_ * self.accum_error_
        
        # Calculate derivative term
        derivative_term = self.Kd_ * (error - self.previous_error_) / dt
        
        # Calculate control command
        output = proportion_term + integral_term + derivative_term
        
        # Update control error
        self.previous_error_ = error
            
        return output

In order to vividly present the results, we also need a function to draw an animation.

In [4]:
%matplotlib
import matplotlib.pyplot as plt

'''
Plot result
'''
def animate(ref_x, ref_y, actual_x, actual_y, ego: State, L, area):
    actual_x.append(ego.pose_.x_)
    actual_y.append(ego.pose_.y_)
    
    plt.cla()
    
    plt.plot(ref_x, ref_y, label='reference path')
    
    half = len(actual_x) // 2
    plt.plot(actual_x[half:], actual_y[half:], label='history path')
    
    plt.xlim(ego.pose_.x_ - area, ego.pose_.x_ + area)
    plt.ylim(ego.pose_.y_ - area, ego.pose_.y_ + area)
    
    front_x = ego.pose_.x_ + L * math.cos(ego.pose_.theta_)
    front_y = ego.pose_.y_ + L * math.sin(ego.pose_.theta_)
    plt.plot([ego.pose_.x_, front_x], [ego.pose_.y_, front_y], label='vehicle body', linewidth=4)
    plt.scatter(ego.pose_.x_, ego.pose_.y_, label='reax axle center')
    
    plt.title("v[km/h]:" + str(ego.v_ * 3.6)[0:4])
    plt.grid(True)
    plt.legend()
    plt.box(True)
    plt.pause(0.01)

Using matplotlib backend: Qt5Agg


**Step 1** Read the planned results stored in the `CSV` file through `numpy` and display them using `Matplotlib`.


In [5]:
import numpy as np
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap

# Read the csv file with numpy's genfromtxt and skip the first row
trajectory_data = np.genfromtxt('./data/global_trajectory.csv', delimiter=';', skip_header=1)

# To avoid errors caused by confusion between row vectors and column vectors, use np.mat instead of np.array for array operations as much as possible
# So here the read data is converted to the format of np.mat
reference_trajectory = np.mat(trajectory_data)

# Split the array for easy operation later
reference_s     = reference_trajectory[:, 0]             # [m]
reference_x     = reference_trajectory[:, 1]             # [m]
reference_y     = reference_trajectory[:, 2]             # [m]
reference_theta = reference_trajectory[:, 3] + math.pi/2 # [rad]
reference_v     = reference_trajectory[:, 5] * 0.44704   # [m/s]

# Use pyplot's Normalize function to normalize the minimum and maximum values of the vehicle speed
# Make its value between 0-1
norm = plt.Normalize(reference_v.min(), reference_v.max())

# Define a dictionary to describe the process of color mapping. In this dictionary, the key is the color channel ('red', 'green', 'blue'), and the value is a list of tuples.
# Each tuple represents the change of a color channel in the color mapping, the first element of the tuple is the position of the color channel (from 0 to 1), the second and third elements are the color values at this position (also from 0 to 1).
# In this example, we define a color mapping that transitions from blue to red.
cdict = {'red':   ((0.0, 0.0, 0.0), (1.0, 1.0, 1.0)), 
         'green': ((0.0, 0.0, 0.0), (1.0, 0.0, 0.0)), 
         'blue':  ((0.0, 1.0, 1.0), (1.0, 0.0, 0.0))} 

# Create a linear segmented color mapping using the defined color dictionary, the name of the mapping is 'Bl_Rd', and the number of segments is 256
c_m = LinearSegmentedColormap('Rd_Bl', cdict, 256)

# Create a new figure window and set its size to 8x8
fig, ax = plt.subplots(figsize=(8, 8))

# Traverse each point in the reference path, select the color according to the vehicle speed, and draw the corresponding path
for i in range(1, len(reference_x)):
    x = np.array(reference_x[i-1:i+1]).ravel()
    y = np.array(reference_y[i-1:i+1]).ravel()
    c = c_m(norm(np.array(reference_v[i-1:i+1]).ravel()))
    ax.plot(x, y, color=c[0])

# Place a colorbar on the right
sm = cm.ScalarMappable(cmap=c_m, norm=norm)
sm.set_array([])
fig.colorbar(sm, label='Speed [m/s]')
plt.grid(True)
plt.title('Global XY plot of reference path')
plt.xlabel('X [m]')
plt.ylabel('Y [m]')
plt.axis('equal')
plt.savefig('./figs/global_trajectory.png')
plt.close()

**Step 2** Start simulation

In [6]:
import numpy as np
from scipy.spatial import cKDTree

# Initialize KD Tree
reference_points = np.hstack((reference_x, reference_y))
kd_tree = cKDTree(reference_points)

# Initialize vehicle
initial_v     = reference_v[0, 0]
initial_x     = reference_x[0, 0] + 1.0
initial_y     = reference_y[0, 0] + 1.0
initial_theta = reference_theta[0, 0]
wheelbase     = 2

initial_state = State(Transform(initial_x, initial_y, initial_theta), initial_v)
vehicle = Vehicle(wheelbase, initial_state)

# Initialize PID controller
pid = PidController(Kp=1.0, Ki=0.1, Kd=0.001)

# Initialize Stanley controller
stanley = Stanley(k=1)

# Setup simulation
total_sim_time = 100
sample_time    = 0.1
sim_time       = np.arange(0, total_sim_time, sample_time)

# Setup empty list to record the simulation process
actual_x_list     = []
actual_y_list     = []
actual_theta_list = []
actual_v_list     = []
actual_err_list   = []
actual_v_ref_list = []

# Start simulation
target_idx = 0
current_state = initial_state
plt.figure(2)

for i in range(len(sim_time)):
    plt.figure(2)
    animate(reference_x, reference_y, actual_x_list, actual_y_list, current_state, wheelbase, area=15)
    
    # Record the simulation process
    actual_x_list.append(current_state.pose_.x_)
    actual_y_list.append(current_state.pose_.y_)
    actual_theta_list.append(current_state.pose_.theta_)
    actual_v_list.append(current_state.v_)
    
    # Calculate the position of front axle center
    front_x = current_state.pose_.x_ + wheelbase * math.cos(current_state.pose_.theta_)
    front_y = current_state.pose_.y_ + wheelbase * math.sin(current_state.pose_.theta_)
    
    tracking_err, target_idx = kd_tree.query([front_x, front_y])
    
    target_x     = reference_x[target_idx, 0]
    target_y     = reference_y[target_idx, 0]
    target_theta = reference_theta[target_idx, 0]
    target_v     = reference_v[target_idx, 0]
    target_pose  = Transform(target_x, target_y, target_theta)
    
    actual_v_ref_list.append(target_v)
    actual_err_list.append(tracking_err)
    
    # Calculate control command and update vehicle state
    # TODO
    delta = 0
    accel = 0
    current_state = current_state
    
    progress = (i + 1) / len(sim_time)
    bar = "#" * int(progress * 20)
    print("\rProgress: [{:<20}] {:.1f}%".format(bar, progress * 100), end="")
print('\nDone!')

# Plot result
plt.figure(3)

# Create the first subplot
plt.subplot(2, 1, 1)
plt.plot(sim_time, actual_v_ref_list, label='reference speed')
plt.plot(sim_time, actual_v_list, label='actual speed')
plt.ylabel('v [m/s]')
plt.axis('tight')
plt.grid(True)
plt.box(True)
plt.legend()

# Create the second subplot
plt.subplot(2, 1, 2)
plt.plot(sim_time, actual_err_list)
plt.ylabel('lateral error [m/s]')
plt.xlabel('time [s]')
plt.axis('tight')
plt.grid(True)
plt.box(True)

plt.show()


Progress: [####################] 100.0%
Done!
